# **Data Formatting Pipeline**

In [1]:
!git clone https://github.com/OscarMoliina/betterlifebetterhealth.git

Cloning into 'betterlifebetterhealth'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 253 (delta 31), reused 70 (delta 21), pack-reused 155
Receiving objects: 100% (253/253), 89.38 MiB | 25.13 MiB/s, done.
Resolving deltas: 100% (81/81), done.


#### Llegir dades i paquets necessaris:

In [2]:
!pip3 install pyspark
import pyspark
from pyspark.sql import SparkSession
import duckdb
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType


spark = SparkSession.builder\
    .appName("Parquet to DuckDB") \
    .master("local[*]") \
    .config("spark.jars", "/content/betterlifebetterhealth/src/utils/duckdb.jar") \
    .getOrCreate()

# Leer el archivo Parquet
demography_df = spark.read.parquet('/content/betterlifebetterhealth/data/parquet/demography.parquet')
mh_df = spark.read.parquet('/content/betterlifebetterhealth/data/parquet/mental_health.parquet')
society_df = spark.read.parquet('/content/betterlifebetterhealth/data/parquet/society.parquet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a2f96a986912bc4c31f8c227cee1d5713d94ed368263197952e9ec32b66c1a2b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


#### Comprovar types de cada columna dels dataframes:

In [3]:
print(demography_df.dtypes)
print(mh_df.dtypes)
print(society_df.dtypes)

[('NAME', 'string'), ('YR', 'bigint'), ('AREA_KM2', 'bigint'), ('CBR', 'double'), ('CDR', 'double'), ('DEATHS', 'double'), ('E0', 'double'), ('GRR', 'double'), ('IMR', 'double'), ('MEDAGE', 'double'), ('MR0_4', 'double'), ('POP_DENS', 'double'), ('genc standard countries and areas', 'bigint'), ('code', 'string')]
[('index', 'bigint'), ('Entity', 'string'), ('Code', 'string'), ('Year', 'string'), ('Schizophrenia (%)', 'string'), ('Bipolar disorder (%)', 'string'), ('Eating disorders (%)', 'string'), ('Anxiety disorders (%)', 'string'), ('Drug use disorders (%)', 'string'), ('Depression (%)', 'string'), ('Alcohol use disorders (%)', 'string')]
[('Country', 'string'), ('Year', 'bigint'), ('Area (square kilometres)', 'double'), ('Total population', 'double'), ('Population density, pers. per sq. km', 'double'), ('Population aged 0-14, male', 'double'), ('Population aged 0-14, female', 'double'), ('Population aged 15-64, male', 'double'), ('Population aged 15-64, female', 'double'), ('Popula

In [4]:
for column in mh_df.columns:
    if '%' in column:
        mh_df = mh_df.withColumn(column, col(column).cast(DoubleType()))

print(mh_df.dtypes)

[('index', 'bigint'), ('Entity', 'string'), ('Code', 'string'), ('Year', 'string'), ('Schizophrenia (%)', 'double'), ('Bipolar disorder (%)', 'double'), ('Eating disorders (%)', 'double'), ('Anxiety disorders (%)', 'double'), ('Drug use disorders (%)', 'double'), ('Depression (%)', 'double'), ('Alcohol use disorders (%)', 'double')]


### Save Basic

In [5]:
d = {
    demography_df:"demography_df",
    mh_df:"mh_df",
    society_df:'society_df'
}

for k,v in d.items():
    k.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:{v}.db") \
        .option("dbtable", f"{v}") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .save()

### Guardat db a Github:

In [6]:
# Clone the repository
%cd betterlifebetterhealth/data/db

# Remove old files
!rm -rf demography_df.db
!rm -rf society_df.db
!rm -rf mh_df.db

# Copy new files from your local directory to this directory
!cp -R /content/*.db .

/content/betterlifebetterhealth/data/db


In [7]:
#CODI A LA DOCUMENTACIO NO POSAT AQUÍ PER PRIVACITAT DE USUARI

[main d468dfe] Update parquet files
 3 files changed, 0 insertions(+), 0 deletions(-)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 730.94 KiB | 9.02 MiB/s, done.
Total 7 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/OscarMoliina/betterlifebetterhealth.git
   5dc9950..d468dfe  main -> main
